In [2]:
from dotenv import load_dotenv

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent

In [7]:
load_dotenv()

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash",
                             timeout=None)


# Server
# - Handles incoming connections (in this case, via stdio)
# - Handles tool discovery (when clients ask "what tools do you have")
# - Execute tools when requested and returns results
server_params = StdioServerParameters(
    command="python",
    args=["/Users/damien/VisualStudio Workspace/AI-Agents/langgraph_mcp_multiserver/assistant/my_mcp/local_servers/math_server.py"],
)
async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:

        # Initialize the connecion
        await session.initialize()

        # Convert MCP tools to LangChain tools
        tools = await load_mcp_tools(session)

        # Create and run the agent
        agent = create_react_agent(llm, tools)
        agent_response = await agent.ainvoke({"messages": "What's (3 + 5) x 12"})

I0000 00:00:1755693346.290084 3275273 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


In [8]:
for m in agent_response['messages']:
    m.pretty_print()

================================ Human Message =================================

What's (3 + 5) x 12
================================== Ai Message ==================================
Tool Calls:
  add (10cb3179-efb2-46ea-b969-8eaa4e13b318)
 Call ID: 10cb3179-efb2-46ea-b969-8eaa4e13b318
  Args:
    a: 3.0
    b: 5.0
================================= Tool Message =================================
Name: add

8
================================== Ai Message ==================================
Tool Calls:
  multiply (23798cd7-edab-4105-8282-b900cff9fcb2)
 Call ID: 23798cd7-edab-4105-8282-b900cff9fcb2
  Args:
    a: 8.0
    b: 12.0
================================= Tool Message =================================
Name: multiply

96
================================== Ai Message ==================================

(3 + 5) x 12 = 96


In [6]:
from langchain_mcp_adapters.client import MultiServerMCPClient


client = MultiServerMCPClient({
    "math": {
        "command": "python",
        "args": ["/Users/damien/VisualStudio Workspace/AI-Agents/langgraph_mcp_multiserver/assistant/my_mcp/local_servers/math_server.py"],
        "transport": "stdio",
    },
    "weather": {
        "command": "python",
        "args": ["/Users/damien/VisualStudio Workspace/AI-Agents/langgraph_mcp_multiserver/assistant/my_mcp/local_servers/weather_server.py"],
        "transport": "stdio",
    },
})

agent = create_react_agent(llm, await client.get_tools())
math_response = await agent.ainvoke({"messages": "What's (3 + 5) x 12?"})
weather_response = await agent.ainvoke({"messages": "What's the weather in nyc?"})

for m in weather_response['messages']:
    m.pretty_print()

I0000 00:00:1755693313.846131 3275273 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1755693313.855247 3275273 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1755693315.612578 3275273 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1755693316.644469 3275273 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1755693321.332000 3275273 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


================================ Human Message =================================

What's the weather in nyc?
================================== Ai Message ==================================
Tool Calls:
  get_forecast (10454d88-810d-43af-b565-e3064a9b203f)
 Call ID: 10454d88-810d-43af-b565-e3064a9b203f
  Args:
    city_name: New York
================================= Tool Message =================================
Name: get_forecast

Weather in New York (40.71427, -74.00597):
- Temperature: 20.3°C
- Wind speed: 13.1 km/h
- Weather code: 3
================================== Ai Message ==================================

The weather in New York is 20.3°C with a wind speed of 13.1 km/h. The weather code is 3.
